In [3]:
#import libraries
from osgeo import gdal, osr
import glob

In [4]:
#create function to convert array into GeoTIFF raster
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    
    rows = array.shape[0]
    cols = array.shape[1]
    print ("cols and rows: ", cols, rows)
    
    originX, pixelWidth, b, originY, d, pixelHeight= dataset.GetGeoTransform()
    print ("originX, pixelWidth, b, originY, d, pixelHeight:",originX, pixelWidth, b, originY, d, pixelHeight)

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    band_num = 1

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX,pixelWidth, 0, originY, 0, pixelHeight))
    originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut = outRaster.GetGeoTransform()
    print ("originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut:",originXOut, pixelWidthOut, bOut, originYOut, dOut, pixelHeightOut)
    
    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[b,:,:])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [ ]:
#define paths
path = r'C:/.../*.tif'#path of MCD64A1 images containing one band (forestry fire dates)
TMPpath="C:/.../"#temporary folder for processing and save output results

#create list of MCD64A1 GeoTIFFs
files = glob.glob(path)
print(files)

In [ ]:
#count the number of times a pixel has been detected as burned and export to GeoTIFF

#create empty array from the first file in the list
TMPfilename = TMPpath+"empty.tif"#create a new filename for a copy of the first image
driver = gdal.GetDriverByName('GTiff')
dst_ds = driver.CreateCopy(TMPfilename, gdal.Open(files[0]), 0 )
ds = gdal.Open(TMPfilename)
b1 = ds.GetRasterBand(1)
emptyArr = b1.ReadAsArray()*0
        
#open image in list 
for file in files:
    
    #create an array from image of interest
    ds1 = gdal.Open(file)
    b1 = ds1.GetRasterBand(1)
    arr = b1.ReadAsArray()
    print("Processing file: "+file)
    rows = arr.shape[0]
    cols = arr.shape[1]
    #check values on each pixel
    for row in range(rows):
        for col in range(cols):
            #if pixel value (date) is superior to 0, count it as 1 and sum with other pixel values superior to 0 from other dates
            if arr[row][col]>0:
                emptyArr[row][col]=emptyArr[row][col]+1
    ds1 = None 
#convert emptyArr to GeoTIFF
array2raster(TMPpath+"sum_burned_areas.tif",gdal.Open(files[0]), emptyArr, "Float32")
ds = None        